In [1]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            if 'damage' in action:
                for damage in action['damage']:
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[1][0])
            
            
            if 'effects' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['effects']=[]

            if 'effects' in action:
                if len(action['effects'])>0:
                    for r in action['effects']:
                        static[str(action['class']) + '_' + str(action['id'])]['effects'].append(r[1])
        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]
            
            
    for k in static.keys():
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        static[k]['各类效果']= list(set(num for sublist in static[k]['effects'] for num in sublist))
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']
        
        
        #隐藏具体伤害
        static[k].pop('damage')
        static[k].pop('effects')

        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    return static


In [2]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T

        print(log_file)
        display(df)

for_qiangye.json


,BOTTOM,LEFT,MOVE_START,RIGHT,TOP,WAIT,各类效果,平均伤害,总伤害,总伤害次数,总移动次数,SKILL_77,最终HP,SKILL_88,SKILL_89,SKILL_90,SKILL_137,EFFECT_64,SKILL_100,SKILL_96,SKILL_80
monster_16714666,0,0,90,3,4,87,[],0,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_616666,25,10,90,4,30,24,"[65, 67]",0.0,0,3,69,3,248.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_1316666,6,3,90,8,18,78,[],0,0,0,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_7115666,0,0,90,2,4,88,[],0,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5003,91,9,71,16,94,NaN,[],81.1,811.0,10,210,NaN,NaN,2,4,4,NaN,NaN,NaN,NaN,NaN
monster_716666,2,0,2,4,0,NaN,"[65, 67]",0.0,0,2,6,NaN,0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
hero_5001,69,17,60,17,69,NaN,[],76.705,613.64,8,172,NaN,NaN,NaN,NaN,NaN,NaN,60,4,2,NaN
hero_5002,72,3,55,15,72,NaN,[],80.333333,482,6,162,NaN,NaN,NaN,NaN,NaN,NaN,55,NaN,NaN,6
monster_13713666,1,1,54,6,4,48,[],0,0,0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_13712666,1,4,54,7,6,47,[],0,0,0,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [382]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            if actions[i]['action_type']=='SKILL_137':
                print(actions[i],states[i])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(99)+".json"
log_path = log_files_path + "/for_qiangye.json"

parse_result_json_detail(log_path)

{'action_type': 'SKILL_137', 'atk_range': [[7, 1, 4]], 'atk_position': [7, 1, 4], 'release_range': [[6, 1, 5], [7, 1, 4], [7, 1, 6], [8, 1, 3], [8, 1, 7], [9, 1, 2], [9, 1, 8], [10, 1, 3], [10, 1, 7], [11, 1, 4], [12, 1, 5]], 'damage': [[['hero', 5003], [0], [-72], ['Equal']]], 'effects': [[['hero', 5003], [78], []]], 'id': 716666, 'class': 'monster'} []
{'action_type': 'SKILL_137', 'atk_range': [[7, 1, 3]], 'atk_position': [7, 1, 3], 'release_range': [[6, 1, 4], [7, 1, 3], [7, 1, 5], [8, 1, 2], [8, 1, 6], [9, 1, 7], [10, 1, 2], [11, 1, 3], [11, 1, 5], [12, 1, 4]], 'damage': [[['hero', 5002], [0, 0], [-67.75, -67.75], ['Equal', 'Equal']]], 'effects': [[['hero', 5002], [67, 65], []]], 'id': 816666, 'class': 'monster'} []
{'action_type': 'SKILL_137', 'atk_range': [[7, 1, 4]], 'atk_position': [7, 1, 4], 'release_range': [[3, 1, 6], [4, 1, 5], [4, 1, 7], [5, 1, 4], [5, 1, 8], [6, 1, 3], [6, 1, 9], [7, 1, 4], [8, 1, 5], [8, 1, 7], [9, 1, 6]], 'damage': [[['hero', 5003], [0], [-72], ['Equal'

In [3]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if  v['stepname']=='决策树-首次查找最终选择' or  v['stepname']=='决策树-多次查找最终选择':
                   #if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                        print(k,v)

In [4]:

tmp_log_file = os.getcwd()+"/../log/logs.json"
parse_decision_log(tmp_log_file)

2024-09-29 11:08:09.335075_6 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'}
2024-09-29 11:08:09.692255_22 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '移动到敌人附近'}
2024-09-29 11:08:10.020094_36 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'}
2024-09-29 11:08:10.365290_50 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'}
2024-09-29 11:08:11.843031_78 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击'}
2024-09-29 11:08:12.502633_99 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击'}
2024-09-29 11:08:13.063448_117 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'}
2024-09-29 11:08:13.524271_131 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'}
2024-09-29 11:08:13.984408_145 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '逃跑'}
2024-09-29 11:08:14.405291_159 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '逃跑'}
2024-09-29 11:08:14.654486_169 {'stepname': '决策树-首次查找最终选